# Pandas確認課題

このPandas確認問題は、データサイエンス100本ノックの問題で最低限必要な問題を抜粋したものですが、[Introduction_to_Pandas](./11_Introduction_to_Pandas.ipynb) に掲載されていない機能が必要な問題もあります。
初めて触るライブラリを調べながら使うというのはよくある光景です。この課題では皆さんにもそれに挑戦していただきます。  
ヒントとして検索キーワードなどを載せておくので、自力で調べながら解いてみましょう。  



## 必要モジュールのインポート

この問題で使うモジュールをインポートします．

In [ ]:
import pandas as pd
import numpy as np

## データの読み込み

In [ ]:
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

---
## 問1. 条件抽出
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [ ]:
import pandas as pd

# データの読み込み
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

# 条件を満たすデータの抽出
filtered_df = df_receipt[
    (df_receipt['customer_id'] == 'CS018205000001') & 
    ((df_receipt['amount'] >= 1000) | (df_receipt['quantity'] >= 5))
]

# 必要な列の指定
result_df = filtered_df[['sales_ymd', 'customer_id', 'product_cd', 'quantity', 'amount']]

# インデックス番号の追加
result_df.reset_index(drop=True, inplace=True)


# 結果の表示
print(result_df)


---
## 問2. ソート
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭5件を全項目表示せよ。

In [ ]:
import pandas as pd

# データの読み込み
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
# データフレームを作成
df_customer = pd.DataFrame(data)
# 生年月日で若い順にソート
df_customer_sorted = df_customer.sort_values(by='birth_day', ascending=False)

# 先頭5件を全項目表示
print(df_customer_sorted.head(5))


---
## 問3. 全件数
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [ ]:
import pandas as pd

# データフレームの読み込み
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

# 全体の件数をカウント
total_count = df_receipt.shape[0]

print(f"データフレームの件数: {total_count}")



## 問4. ユニーク件数
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [ ]:
import pandas as pd

# データフレームの読み込み
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

# 顧客IDのユニークな件数をカウント
unique_customer_count = df_receipt['customer_id'].nunique()

print(f"ユニークな顧客IDの件数: {unique_customer_count}")



<details>
<summary>ヒント</summary>
「ユニーク」というのはそのまま検索に使える単語です。  
</details>

---
## 問5. 〇〇ごとに集計
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは先頭5件だけ表示せよ。

会員のみを抽出する方法は、例えば以下の2通りの方法があります。

In [ ]:
df_receipt_only_member = df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
df_receipt_only_member = df_receipt.query("not customer_id.str.startswith('Z')", engine="python")

In [ ]:
import pandas as pd
# データフレームの読み込み
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

# 顧客IDが "Z" から始まるものを除外
df_filtered = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]

# 顧客IDごとに売上金額を合計
df_sum = df_filtered.groupby('customer_id')['amount'].sum().reset_index()

# 全顧客の売上金額の平均を計算
average_amount = df_sum['amount'].mean()

# 平均以上に買い物をしている顧客を抽出
df_above_average = df_sum[df_sum['amount'] >= average_amount]

# 結果を表示（先頭5件）
print(df_above_average.head())


<details>
<summary>ヒント1</summary>
「pandas 要素ごと 集計」 などで今回使える機能に関する記事が見つかります。
</details>

<details>
<summary>ヒント2</summary>
メソッド名は "groupby" です。
</details>

---
## 問6. DataFrameの結合
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は先頭5件だけ表示せよ。

In [ ]:
df_customer_only_member = df_customer[~df_customer["customer_id"].str.startswith("Z")]
df_customer_only_member = df_customer.query("not customer_id.str.startswith('Z')", engine="python")

In [ ]:
import pandas as pd

# データフレームの読み込み
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

# 女性の顧客を対象とし、非会員（顧客IDが 'Z' から始まるもの）を除外
df_female_customers = df_customer[(df_customer['gender_cd'] == 1) & (~df_customer['customer_id'].str.startswith('Z'))]

# レシート明細の顧客IDと売上金額を集計
df_sales = df_receipt.groupby('customer_id')['amount'].sum().reset_index()

# 顧客IDが買い物の実績がない顧客については売上金額を0とするために左結合
df_customer_sales = pd.merge(df_female_customers[['customer_id']], df_sales, on='customer_id', how='left')

# 売上金額がNaNの顧客は0に置き換える
df_customer_sales['amount'].fillna(0, inplace=True)

# 結果を表示（先頭5件）
print(df_customer_sales.head())


<details>
<summary>ヒント1</summary>
タイトル通り 「pandas DataFrame 結合」などと調べれば必要な機能に関する記事が見つかります。  
</details>


<details>
<summary>ヒント2</summary>
"merge", "join"という似たメソッドがあります。  
今回の場合"merge"が便利でしょう。
</details>

---
## 問7. 時系列データ
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。なお、データは先頭5件を表示せよ。

In [ ]:
import pandas as pd

# データフレームの読み込み
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')

# application_date 列を日付型に変換
df_customer['application_date'] = pd.to_datetime(df_customer['application_date'], format='%Y%m%d')

# 顧客IDと申し込み日を抽出
df_result = df_customer[['customer_id', 'application_date']]

# 結果を表示（先頭5件）
print(df_result.head())


<details>
<summary>ヒント1</summary>
「pandas datetime」などで該当の機能が見つかるかと思います。
</details>


<details>
<summary>ヒント2</summary>
"pd.to_datetime"というメソッドが使えるでしょう。このメソッドを適用する際ですが、for文を使わずに実装しましょう。

---
## 問8. 関数
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、結果は先頭5件を表示せよ。

In [ ]:
import pandas as pd
import numpy as np

# データフレームの読み込み
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

# 顧客IDが "Z" から始まるものを除外
df_filtered = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]

# 顧客IDごとに売上金額を合計
df_sum = df_filtered.groupby('customer_id')['amount'].sum().reset_index()

# 合計売上金額の常用対数を計算
df_sum['log_amount'] = np.log10(df_sum['amount'])

# 結果を表示（先頭5件）
print(df_sum.head())


---
## 問9. 欠損数
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [ ]:
import pandas as pd

# データフレームの読み込み
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')

# 各項目の欠損値の数を確認
missing_counts = df_product.isnull().sum()

print("各項目の欠損値の数:")
print(missing_counts)


---
## 問10. 欠損値の除去
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [ ]:
import pandas as pd

# データフレームの読み込み
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')

# 削除前の件数を表示
count_before = df_product.shape[0]
print(f"削除前の件数: {count_before}")

# 欠損値を含むレコードを削除
df_product_1 = df_product.dropna()

# 削除後の件数を表示
count_after = df_product_1.shape[0]
print(f"削除後の件数: {count_after}")

# 削除前後の件数の差を確認
deleted_count = count_before - count_after
print(f"削除されたレコード数: {deleted_count}")


In [ ]:
len(df_product), len(df_product_1)

---
## 問11. 欠損値の穴埋め
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とせよ。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
import pandas as pd
import numpy as np

# 例としてのデータフレームを作成
df_product = pd.DataFrame({
    'product_cd': [1, 2, 3, 4, 5],
    'category_major_cd': ['A', 'B', 'A', 'C', 'B'],
    'category_medium_cd': ['A1', 'B1', 'A2', 'C1', 'B2'],
    'category_small_cd': ['A1a', 'B1a', 'A2a', 'C1a', 'B2a'],
    'unit_price': [100, np.nan, 200, np.nan, 150],
    'unit_cost': [50, 60, np.nan, 40, np.nan]
})

# unit_price と unit_cost の欠損値を平均値で補完する
mean_unit_price = df_product['unit_price'].mean()
mean_unit_cost = df_product['unit_cost'].mean()

# 1円未満を四捨五入
mean_unit_price = round(mean_unit_price)
mean_unit_cost = round(mean_unit_cost)

# 欠損値を平均値で補完する
df_product_2 = df_product.copy()
df_product_2['unit_price'].fillna(mean_unit_price, inplace=True)
df_product_2['unit_cost'].fillna(mean_unit_cost, inplace=True)

# 欠損値がないことを確認する
print(df_product_2.isnull().sum())


### 余談
ChatGPTやBing AIに聞けば大抵のことは教えてくれます。  
何回か入力文章を吟味しないといけないこともありますが、知らないことを調べる場合は自分で検索するよりも早いです。  
ただ、ChatGPTなどは嘘をつく場合があるので、自分でソースを参照する姿勢は必要です。  

これはBingAIの回答例です。  

![BingAIの回答例](./imgs/pandas/BingAI.png)